In [ ]:
from funconnect import ta3, nda
from funconnect.morphology import schema
import datajoint as dj

from scipy import stats
import numpy as np

@schema
class FunCorrelation(dj.Computed):
    definition = """
    # correlation of traces for connected pairs of cells
    (scan_idx, pre_slice, pre_id) -> nda.Trace(scan_idx, slice, em_id)
    (post_slice, post_id) -> nda.Trace(slice, em_id)
    ---
    correlation          : float      # Pearson's correlation coefficient
    p_val                : float      # two-tailed p_value of slope of the linear regression
    """
    
    @property
    def cell_synapses(self):
        presoma = ta3.Soma().proj(pre_id='em_id', presyn='segment_id')
        postsoma = ta3.Soma().proj(post_id='em_id', postsyn='segment_id')
        return presoma * postsoma & ta3.Synapse
    
    @property
    def key_source(self):
        pres = nda.Trace.proj(pre_slice='slice', pre_id='em_id')
        posts = nda.Trace.proj(post_slice='slice', post_id='em_id')
        return pres * posts & 'pre_id != post_id'
        
    def make(self, key):
        pre = dict(key)
        pre['slice'] = pre['pre_slice']
        pre['em_id'] = pre['pre_id']
        pre_trace = (nda.Trace & pre).fetch1('trace').reshape(-1)
        pre_trace.shape
        post = dict(key)
        post['slice'] = post['post_slice']
        post['em_id'] = post['post_id']
        post_trace = (nda.Trace & post).fetch1('trace').reshape(-1)
        
        correlation = stats.pearsonr(pre_trace, post_trace)[0]
        p_val = stats.pearsonr(pre_trace, post_trace)[1]
        
        key['correlation'] = correlation
        key['p_val'] = p_val 
        self.insert1(key)

synapses_funcorr = FunCorrelation() & FunCorrelation().cell_synapses
        

In [ ]:
dj.conn()

In [ ]:
FunCorrelation()

In [ ]:
keys = FunCorrelation().key_source.fetch('KEY')

In [ ]:
key = keys[0]
key

In [ ]:
stats.pearsonr?

In [ ]:
pre = dict(key)
pre['slice'] = pre['pre_slice']
pre['em_id'] = pre['pre_id']
pre_trace = (nda.Trace & pre).fetch1('trace').reshape(-1)
pre_trace.shape
post = dict(key)
post['slice'] = post['post_slice']
post['em_id'] = post['post_id']
post_trace = (nda.Trace & post).fetch1('trace').reshape(-1)
from scipy import stats
corr = stats.pearsonr(pre_trace, post_trace)[0]
p_val = stats.pearsonr(pre_trace, post_trace)[1]

In [ ]:
p_val

In [ ]:
synapses_funcorr

In [ ]:
FunCorrelation().key_source

In [ ]:
FunCorrelation().key_source

In [ ]:
k = FunCorrelation()

In [ ]:
k - k.cell_synapses

In [ ]:
FunCorrelation.populate(FunCorrelation().cell_synapses, display_progress=True)

In [ ]:
FunCorrelation() & FunCorrelation().cell_synapses

In [ ]:
FunCorrelation() & FunCorrelation().cell_sy

In [ ]:
FunCorrelation().

In [ ]:
@schema
class FunCorrelation(dj.Computed):
    definition = """
    # correlation of traces for connected pairs of cells
    -> nda.Scan
    (pre_slice, pre_id) -> nda.Mask(slice, em_id)
    (post_slice, post_id) -> nda.Mask(slice, em_id)
    ---
    correlation          : float      # correlation of traces
    """
    key_source = pres * posts & cell_syn
    segments = ta3.Segment & (ta3.Soma & key) # All the segments containing the postsyn cell
    postCells = (ta3.Soma & key) # All the postsyn cells
    preCells = ta3.Soma.proj(presyn='segment_id') & (ta3.Synapse * segments.proj(postsyn='segment_id'))
    
    scans = nda.Scan & (nda.Trace & key)
    pretraces = (nda.Trace & preCells & scans).proj(preslice='slice', preid='em_id', pretrace='trace')
    posttraces = (nda.Trace & key).proj(postslice='slice', postid='em_id', posttrace='trace')
   
    def _make_tuples(self, key):
        print(key)

In [ ]:
FunCorrelation.populate()